In [14]:
import csv
import math
import os
import pandas as pd
import pathlib
import re

In [15]:
list_extractor = {
    'lbp': [59],
    'surf64': [128, 256, 257],
    'surf128': [128, 256, 513],
    'mobilenetv2': [128, 256, 512, 1024, 1280],
    'resnet50v2': [128, 256, 512, 1024, 2048],
    'vgg16': [128, 256, 512]
}
list_classifier = ['DecisionTreeClassifier', 'KNeighborsClassifier', 'MLPClassifier', 'RandomForestClassifier', 'SVC']
list_dim = [256, 400, 512]
list_metrics = ['mean', 'std']
list_source = ['manual', 'unet']
list_time = ['search_best_params', 'train_valid']

path = 'resultados'

In [16]:
columns = [c + '_' + str(d) + '_' + s for c in list_classifier for s in sorted(list_source) for d in list_dim]
# columns = list(filter(lambda x: not('400_manual' in x) and not('512_manual' in x), columns))

index = [e + '_' + str(d) + '_' + m for e in list_extractor.keys() for d in reversed(list_extractor[e]) for m in list_metrics]
index_folder = [e + '_' + str(d) for e in list_extractor.keys() for d in reversed(list_extractor[e])]
index_time = [e + '_' + str(d) + '_' + t for e in list_extractor.keys() for d in reversed(list_extractor[e]) for t in sorted(list_time, reverse=True)]

In [17]:
df = pd.DataFrame(columns=columns, index=index)
df_folder = pd.DataFrame(columns=columns, index=index_folder)
df_time = pd.DataFrame(columns=columns, index=index_time)

In [32]:
def file_is_colormode_and_slice_and_patch(file):
    data_info = pd.read_csv(os.path.join(str(file).replace('mean.csv', 'info.csv')), sep=";", header=None, index_col=0)
    color_mode = data_info.loc['color_mode'][1]
    dim = data_info.loc['dim_image'][1]
    n_patch = data_info.loc['n_patch'][1]
    slice = data_info.loc['slice'][1]
    return True if color_mode.lower() == 'grayscale' and ('SEM_RESIZE' != dim) and (slice == 'horizontal' or math.isnan(slice)) and (n_patch == '3' or math.isnan(n_patch)) else False

list_files = [p for p in pathlib.Path(path).rglob('mean.csv') if p.is_file()]
list_files = [file for file in list_files if file_is_colormode_and_slice_and_patch(file)]
len(list_files)

640

In [33]:
for file in list_files:
    _, _, date, source, dim_image, extractor, classifier, _, n_features, _ = re.split('/', str(file))
    data = pd.read_csv(file, sep=";", header=None, index_col=0).squeeze()
    index = extractor + '_' + str(n_features)
    column = classifier + '_' + dim_image + '_' + source

    index_mean = index + '_mean'
    index_std = index + '_std'
    df.loc[index_mean, column] = str(f'=ROUND({data.loc["mean_sum"][1]} * 100, 2)')
    df.loc[index_std, column] = str(f'=\"±\"&ROUND({data.loc["std_sum"][1]} * 100, 2)')

    df_folder.loc[index, column] = date

    index_mean_time_train_valid = index + '_train_valid'
    index_mean_time_search_best_params = index + '_search_best_params'
    df_time.loc[index_mean_time_train_valid, column] = str(f'=ROUND({data.loc["mean_time_train_valid"][1]}, 2)')
    df_time.loc[index_mean_time_search_best_params, column] = str(f'=ROUND({data.loc["mean_time_search_best_params"][1]}, 2)')
df

,DecisionTreeClassifier_256_manual,DecisionTreeClassifier_400_manual,DecisionTreeClassifier_512_manual,DecisionTreeClassifier_256_unet,DecisionTreeClassifier_400_unet,DecisionTreeClassifier_512_unet,KNeighborsClassifier_256_manual,KNeighborsClassifier_400_manual,KNeighborsClassifier_512_manual,KNeighborsClassifier_256_unet,...,RandomForestClassifier_512_manual,RandomForestClassifier_256_unet,RandomForestClassifier_400_unet,RandomForestClassifier_512_unet,SVC_256_manual,SVC_400_manual,SVC_512_manual,SVC_256_unet,SVC_400_unet,SVC_512_unet
lbp_59_mean,"=ROUND(0.39466666666666667 * 100, 2)","=ROUND(0.4746666666666666 * 100, 2)","=ROUND(0.536 * 100, 2)","=ROUND(0.4613333333333333 * 100, 2)","=ROUND(0.4613333333333333 * 100, 2)","=ROUND(0.4693333333333333 * 100, 2)","=ROUND(0.48 * 100, 2)","=ROUND(0.5413333333333334 * 100, 2)","=ROUND(0.5279999999999999 * 100, 2)","=ROUND(0.5013333333333334 * 100, 2)",...,"=ROUND(0.5786666666666667 * 100, 2)","=ROUND(0.5306666666666666 * 100, 2)","=ROUND(0.5813333333333334 * 100, 2)","=ROUND(0.5733333333333334 * 100, 2)","=ROUND(0.3253333333333333 * 100, 2)","=ROUND(0.352 * 100, 2)","=ROUND(0.3546666666666667 * 100, 2)","=ROUND(0.3253333333333333 * 100, 2)","=ROUND(0.352 * 100, 2)","=ROUND(0.37066666666666664 * 100, 2)"
lbp_59_std,"=""±""&ROUND(0.03435759660460033 * 100, 2)","=""±""&ROUND(0.05756542171979442 * 100, 2)","=""±""&ROUND(0.04414118761932492 * 100, 2)","=""±""&ROUND(0.04008879034232775 * 100, 2)","=""±""&ROUND(0.05373184447895969 * 100, 2)","=""±""&ROUND(0.04079215610874226 * 100, 2)","=""±""&ROUND(0.06477310827461932 * 100, 2)","=""±""&ROUND(0.05503937984146746 * 100, 2)","=""±""&ROUND(0.058179797944723825 * 100, 2)","=""±""&ROUND(0.054389541478323054 * 100, 2)",...,"=""±""&ROUND(0.04349201714746691 * 100, 2)","=""±""&ROUND(0.041654664938168856 * 100, 2)","=""±""&ROUND(0.06881860213634101 * 100, 2)","=""±""&ROUND(0.05399588461684421 * 100, 2)","=""±""&ROUND(0.07617523657112373 * 100, 2)","=""±""&ROUND(0.081147328291749 * 100, 2)","=""±""&ROUND(0.08541662601625048 * 100, 2)","=""±""&ROUND(0.061737796814881195 * 100, 2)","=""±""&ROUND(0.05998518335574396 * 100, 2)","=""±""&ROUND(0.04249967320135794 * 100, 2)"
surf64_257_mean,"=ROUND(0.5439999999999999 * 100, 2)","=ROUND(0.5066666666666666 * 100, 2)","=ROUND(0.5466666666666666 * 100, 2)","=ROUND(0.5573333333333333 * 100, 2)","=ROUND(0.5573333333333333 * 100, 2)","=ROUND(0.5546666666666666 * 100, 2)","=ROUND(0.4693333333333333 * 100, 2)","=ROUND(0.45066666666666666 * 100, 2)","=ROUND(0.3413333333333333 * 100, 2)","=ROUND(0.472 * 100, 2)",...,"=ROUND(0.6666666666666667 * 100, 2)","=ROUND(0.696 * 100, 2)","=ROUND(0.7066666666666667 * 100, 2)","=ROUND(0.6586666666666667 * 100, 2)","=ROUND(0.5013333333333334 * 100, 2)","=ROUND(0.36000000000000004 * 100, 2)","=ROUND(0.30666666666666664 * 100, 2)","=ROUND(0.3786666666666667 * 100, 2)","=ROUND(0.33866666666666667 * 100, 2)","=ROUND(0.21333333333333332 * 100, 2)"
surf64_257_std,"=""±""&ROUND(0.08149710287965933 * 100, 2)","=""±""&ROUND(0.07495183638696103 * 100, 2)","=""±""&ROUND(0.02796823595120404 * 100, 2)","=""±""&ROUND(0.04414118761932494 * 100, 2)","=""±""&ROUND(0.06276587891167906 * 100, 2)","=""±""&ROUND(0.03827386691841951 * 100, 2)","=""±""&ROUND(0.04079215610874228 * 100, 2)","=""±""&ROUND(0.052931821976744536 * 100, 2)","=""±""&ROUND(0.03827386691841953 * 100, 2)","=""±""&ROUND(0.05631656555184767 * 100, 2)",...,"=""±""&ROUND(0.05719362823873927 * 100, 2)","=""±""&ROUND(0.06553709721303738 * 100, 2)","=""±""&ROUND(0.049888765156985884 * 100, 2)","=""±""&ROUND(0.058787753826796275 * 100, 2)","=""±""&ROUND(0.024729649321321896 * 100, 2)","=""±""&ROUND(0.08044321668799125 * 100, 2)","=""±""&ROUND(0.07155417527999328 * 100, 2)","=""±""&ROUND(0.062311047709588924 * 100, 2)","=""±""&ROUND(0.0575654217197944 * 100, 2)","=""±""&ROUND(0.048442405665559865 * 100, 2)"
surf64_256_mean,"=ROUND(0.5439999999999999 * 100, 2)","=ROUND(0.5093333333333334 * 100, 2)","=ROUND(0.47733333333333333 * 100, 2)","

In [34]:
df.to_csv(os.path.join('mean.csv'), sep=';', na_rep='', quoting=csv.QUOTE_ALL)
df.to_excel(os.path.join('mean.xlsx'), na_rep='')
df_folder.to_csv(os.path.join('folder.csv'), sep=';', na_rep='', quoting=csv.QUOTE_ALL)
df_time.to_csv(os.path.join('mean_time.csv'), sep=';', na_rep='', quoting=csv.QUOTE_ALL)
df_time.to_excel(os.path.join('mean_time.xlsx'), na_rep='')
df_time

,DecisionTreeClassifier_256_manual,DecisionTreeClassifier_400_manual,DecisionTreeClassifier_512_manual,DecisionTreeClassifier_256_unet,DecisionTreeClassifier_400_unet,DecisionTreeClassifier_512_unet,KNeighborsClassifier_256_manual,KNeighborsClassifier_400_manual,KNeighborsClassifier_512_manual,KNeighborsClassifier_256_unet,...,RandomForestClassifier_512_manual,RandomForestClassifier_256_unet,RandomForestClassifier_400_unet,RandomForestClassifier_512_unet,SVC_256_manual,SVC_400_manual,SVC_512_manual,SVC_256_unet,SVC_400_unet,SVC_512_unet
lbp_59_train_valid,"=ROUND(0.004691171646118164, 2)","=ROUND(0.013353204727172852, 2)","=ROUND(0.013595104217529297, 2)","=ROUND(0.009168052673339843, 2)","=ROUND(0.013418960571289062, 2)","=ROUND(0.004778432846069336, 2)","=ROUND(0.016124582290649413, 2)","=ROUND(0.016391706466674805, 2)","=ROUND(0.01395106315612793, 2)","=ROUND(0.014387941360473633, 2)",...,"=ROUND(0.29349775314331056, 2)","=ROUND(0.8183210849761963, 2)","=ROUND(1.1349491119384765, 2)","=ROUND(0.851383113861084, 2)","=ROUND(0.01819334030151367, 2)","=ROUND(0.01818089485168457, 2)","=ROUND(0.01814098358154297, 2)","=ROUND(0.017798614501953126, 2)","=ROUND(0.018107080459594728, 2)","=ROUND(0.018214941024780273, 2)"
lbp_59_search_best_params,"=ROUND(0.6917109489440918, 2)","=ROUND(0.7119910717010498, 2)","=ROUND(0.06824374198913574, 2)","=ROUND(0.6977121829986572, 2)","=ROUND(0.0630035400390625, 2)","=ROUND(0.05117630958557129, 2)","=ROUND(0.09612011909484863, 2)","=ROUND(0.0957634449005127, 2)","=ROUND(0.09520864486694336, 2)","=ROUND(0.09614038467407227, 2)",...,"=ROUND(19.477655172348022, 2)","=ROUND(19.91133213043213, 2)","=ROUND(20.243176460266113, 2)","=ROUND(19.946508646011353, 2)","=ROUND(0.10139679908752441, 2)","=ROUND(0.09725236892700195, 2)","=ROUND(0.09712433815002441, 2)","=ROUND(0.10305380821228027, 2)","=ROUND(0.09100961685180664, 2)","=ROUND(0.09554600715637207, 2)"
surf64_257_train_valid,"=ROUND(0.04181761741638183, 2)","=ROUND(0.006405496597290039, 2)","=ROUND(0.027534627914428712, 2)","=ROUND(0.040030288696289065, 2)","=ROUND(0.041680192947387694, 2)","=ROUND(0.04212055206298828, 2)","=ROUND(0.014233064651489259, 2)","=ROUND(0.014344406127929688, 2)","=ROUND(0.01669163703918457, 2)","=ROUND(0.015107154846191406, 2)",...,"=ROUND(0.54923095703125, 2)","=ROUND(0.780180549621582, 2)","=ROUND(1.3338435173034668, 2)","=ROUND(0.5504807472229004, 2)","=ROUND(0.20834150314331054, 2)","=ROUND(0.0415135383605957, 2)","=ROUND(0.042166709899902344, 2)","=ROUND(0.039708518981933595, 2)","=ROUND(0.04129114151000977, 2)","=ROUND(0.0452115535736084, 2)"
surf64_257_search_best_params,"=ROUND(0.17948460578918457, 2)","=ROUND(0.15362000465393066, 2)","=ROUND(0.16054606437683105, 2)","=ROUND(0.17825031280517578, 2)","=ROUND(0.21343469619750977, 2)","=ROUND(0.18464064598083496, 2)","=ROUND(0.12561607360839844, 2)","=ROUND(0.1247718334197998, 2)","=ROUND(0.1189572811126709, 2)","=ROUND(0.12685322761535645, 2)",...,"=ROUND(24.51569104194641, 2)","=ROUND(24.41790771484375, 2)","=ROUND(25.026521682739258, 2)","=ROUND(24.567867040634155, 2)","=ROUND(0.10853099822998047, 2)","=ROUND(0.13368797302246094, 2)","=ROUND(0.12474966049194336, 2)","=ROUND(0.12960219383239746, 2)","=ROUND(0.1226053237915039, 2)","=ROUND(0.10950136184692383, 2)"
surf64_256_train_valid,"=ROUND(0.041780710220336914, 2)","=ROUND(0.0424860954284668, 2)","=ROUND(0.041710186004638675, 2)","=ROUND(0.027914810180664062, 2)","=ROUND(0.02762713432312012, 2)","=ROUND(0.04327397346496582, 2)","=ROUND(0.017299079895019533, 2)","=ROUND(0.01619405746459961, 2)","=ROUND(0.016883039474487306, 2)","=ROUND(0.01701216697692871, 2)",...,"=ROUND(1.3437564373016357, 2)","=ROUND(1.3281028747558594, 2)","=ROUND(0.7788416385650635, 2)","=ROUND(1.1036677360534668, 2)","=ROUND(0.2090759754180908, 2)","=ROUND(0.040790653228759764, 2)","=ROUND(0.041366958618164064, 2)","=ROUND(0.03885211944580078, 2)","=ROUND(0.040053272247314455, 2)","=ROUND(0.04497294425964356, 2)"
surf64_256_search_best_pa